In [1]:
import sys
sys.path.append('../')

from utilities.transitionDatasetGeneration import *

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [4]:
maze1 = {"name":"maze1",
        "layout":[
        ['x', '#', 'x', 'x', '.', '.', '.', '.', '#', '#', '#', '#', '.', 'G'],#0
        ['x', '#', 'x', '#', '#', '#', '.', '.', '.', '.', '.', '.', '.', '.'],#1
        ['x', 'x', 'x', 'x', '.', '#', '.', '.', '#', '.', '.', '#', '.', '.'],#2
        ['x', '#', '#', '#', '.', '#', '.', '.', '.', '.', '.', '#', '.', '.'],#3
        ['x', 'x', '#', 'x', '.', '#', '#', '#', '#', '#', '.', '#', '#', '.'],#4
        ['x', 'x', 'x', 'x', '.', '.', '.', 'x', 'x', 'x', '.', '.', '#', '.'],#5
        ['#', '#', '#', '#', '#', '.', '#', '#', 'x', 'x', '.', '.', '#', '.'],#6
        ['.', 'x', 'x', 'x', '#', '.', '.', '#', 'x', 'x', '.', '.', '.', '.'],#7
        ['.', 'x', 'x', 'x', '#', '.', '.', '#', '.', '#', '.', '#', '#', '#'],#8
        ['#', 'x', '#', '.', '#', '.', '#', '#', '.', '#', '.', '.', '.', '.'],#9
        ['.', '.', '#', '.', '.', '.', '#', '.', '.', '.', '.', '.', '.', '.'],#10
        ['.', '#', '#', '.', '#', '.', '.', '.', '.', '#', '.', '#', '#', '.'],#11
        ['.', '.', '.', '.', '#', '#', '.', '#', '.', '#', '.', '.', '#', '.'],#12
        ['S', '.', '.', '.', '#', '.', '.', '#', '.', '#', '.', '.', '#', '.'],#13
        # 0    1    2    3    4    5    6    7    8    9   10   11   12   13
        ]}

maze2 = {"name":"maze2",
        "layout":[
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],#0
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.', '.'],#1
        ['#', '#', '.', '#', '.', '.', '.', '.', '.', '#', '#', '#', '#', '.'],#2
        ['#', '.', '.', '#', '#', '#', '#', '#', '#', '#', '.', '.', '.', '.'],#3
        ['.', '.', '.', '#', 'x', 'x', 'x', 'x', 'x', '#', '.', '#', '.', '.'],#4
        ['.', '.', '.', '#', 'x', 'x', 'x', 'x', 'x', '#', '.', '.', '.', '.'],#5
        ['.', '#', '.', '.', 'x', 'x', 'x', 'x', 'x', '#', '.', '.', '.', '.'],#6
        ['.', '#', '.', '#', 'x', 'x', 'x', 'x', 'x', '#', '.', '.', '.', '.'],#7
        ['.', '.', '.', '#', 'x', 'x', 'x', 'x', 'x', '#', '.', '.', '.', '.'],#8
        ['.', '#', '.', '#', '#', '#', '.', '#', '#', '#', '.', '.', '.', '.'],#9
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '#', '.'],#10
        ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '#', '.'],#11
        ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.', '#', '.'],#12
        ['.', '.', '.', '.', '.', '.', '.', '.', 'x', '.', '.', '.', '.', '.'],#13
        # 0    1    2    3    4    5    6    7    8    9   10   11   12   13
        ]}

maze3 = {"name":"maze3",
        "layout":[
        ['.', '.', 'G'],#0
        ['.', '.', '.'],#1
        ['x', 'x', 'x'],#2
        # 0    1    2
        ]}

In [5]:
maze = maze1["layout"]
n_epochs = 100000
n_models = 5
hidden_size = 512

train_transitions, val_transitions = transition_generator(maze)

In [9]:
models_arr = []

for i in range(n_models):
    print(f"Training model {i+1}...")

    model = ModelTrainer(train_transitions, val_transitions, n_epochs=120000, rand_seed=True)

    model.fit()
    models_arr.append(model)

Training model 1...


100%|██████████| 120000/120000 [27:03<00:00, 73.91it/s]


Trained successfully!
Final loss: 0.007205675821751356

Training model 2...


100%|██████████| 120000/120000 [28:10<00:00, 70.97it/s]


Trained successfully!
Final loss: 0.01581323705613613

Training model 3...


...

Trained successfully!
Final loss: 0.00015583621279802173

Training model 4...


100%|██████████| 120000/120000 [28:43<00:00, 69.62it/s]


Trained successfully!
Final loss: 0.0002739924530033022

Training model 5...


100%|██████████| 120000/120000 [28:46<00:00, 69.49it/s]

Trained successfully!
Final loss: 0.000992210116237402



In [13]:
for i, model in enumerate(models_arr):
    torch.save(model.model.state_dict(), f'model_{i}.pt')

In [14]:
print('\nEVALUATION:\n')

arr1 = []
arr1_raw = []
arr2 = []
arr2_raw = []


for model in models_arr:
    # Single predictions
    with torch.no_grad():
        # Move the input data to the GPU
        inp1 = torch.FloatTensor([[5, 3, 3]]).to(device)
        inp2 = torch.FloatTensor([[5, 4, 3]]).to(device)
        
        # Perform inference on the GPU
        resultado1 = model.model(inp1)
        resultado2 = model.model(inp2)
        
        # Move the result back to CPU and convert to numpy
        resultado1 = resultado1.cpu().detach().numpy()
        resultado2 = resultado2.cpu().detach().numpy()

        arr1.append(np.int32(np.round(resultado1[0])))
        arr1_raw.append(resultado1[0])
        arr2.append(np.int32(np.round(resultado2[0])))
        arr2_raw.append(resultado2[0])
        
for elem, elem2 in zip(arr1, arr2):
    print(elem, '         ', elem2)

#whats the mean and the std of the raw values?
print('\n')
print('Mean of raw values:')
print(np.mean(arr1_raw, axis=0))
print(np.mean(arr2_raw, axis=0))
print('\n')
print('Std of raw values:')
print(np.std(arr1_raw, axis=0))
print(np.std(arr2_raw, axis=0))
print(np.linalg.norm(np.std(arr1_raw, axis=0)))
print(np.linalg.norm(np.std(arr2_raw, axis=0)))


EVALUATION:

[5 3]           [5 5]
[4 4]           [5 5]
[5 5]           [5 5]
[6 4]           [5 5]
[5 3]           [5 5]


Mean of raw values:
[5.20196   3.9789212]
[5.0196676 5.0015707]


Std of raw values:
[0.4481345 0.8320935]
[0.02923888 0.01761499]
0.94509476
0.034135025
